In [ ]:
#Import Packages
from selenium import webdriver
import time
import pandas as pd
import os
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

#Change the link according to need
url1 = 'https://ph.linkedin.com/jobs/python-programmer-jobs-makati?position=1&pageNum=0'

#Load the web driver and get the url

driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get(url1)

#Find number of job listings
y = driver.find_elements(By.CLASS_NAME, 'results-context-header__job-count')[0].text

print(y)

if y.endswith('+') and "," in y:
    cleaned_y = y[:-1].replace(",", "")
    
else:
    cleaned_y = y

n = pd.to_numeric(cleaned_y)

n 

#Loop to scroll through all jobs and click on see more jobs button for infinite scrolling
i = 2
while i <= int((n+200)/25)+1: 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    
    try:
        send = driver.find_element(By.XPATH, "//button[@aria-label='Makita ang higit pang trabaho']")
        driver.execute_script("arguments[0].click();", send)   
        time.sleep(3)
            
    except:
        pass
        time.sleep(5)
        
#Create empty lists for company name and job title
companyname= []
titlename= []

#Find company name and append it to the blank list
try:
    for i in range(n):
        company=driver.find_elements(By.CLASS_NAME, 'base-search-card__subtitle')[i].text
        companyname.append(company)
    
except IndexError:
    print("no")
    
len(companyname)

try:
    for i in range(n):
        title = driver.find_elements(By.CLASS_NAME, 'base-search-card__title')[i].text
        
        titlename.append(title)
        
except IndexError:
    print('No')


len(titlename)

##n Finding the Job Links
job_list = driver.find_elements(By.CLASS_NAME, 'base-card__full-link')
href_list = []

for item in job_list:
    href_list.append(item.get_attribute('href'))
    

#Using Pandas to Create a Table
company_final = pd.DataFrame(companyname, columns = ["Company"])
title_final = pd.DataFrame(titlename, columns = ["Title"])
#links_list = pd.DataFrame(href_list, columns = ["Job Links"])

scraped_jobs = company_final.join(title_final)

scraped_jobs

#Saving to a file in my directory
scraped_jobs.to_csv('pythonjobs_makati.csv')